In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from copy import deepcopy
from random import randint
import json

In [2]:
df = pd.read_csv('bigdata.csv', names=['id', 'name',
               'comments', 'likes', 'followers', 'description', 'brand_type'])
df.drop_duplicates(inplace=True)


In [3]:
with open('english', 'r') as f:
    stopWords = f.read().split('\n')

In [4]:
engage = df['followers'] / ((df['comments'] + df['likes']) / 2)
df['engage'] = engage
df['index'] = np.arange(len(df))
magicWords = {}  
bagOfWords = {}
listOfPairs = {}

In [5]:
def get_value(description: str, index: int):
    for word in description.split():
        word = word.strip().lower()
        if word not in stopwords.words('english'):
            if bagOfWords.get(word) != None:
                bagOfWords[word] += df['engage'][index]
            else:
                bagOfWords[word] = 0

In [6]:
vectorizer = CountVectorizer(ngram_range = (3, 3))

In [7]:
for brand_type in ['fashion', 'beauty', 'electronics']:
    dfType = df[df['brand_type'] == brand_type]
    dfTypeCopy = deepcopy(dfType)
    arr = dfTypeCopy['description'].to_numpy()
    bag = vectorizer.fit_transform(arr.astype('U'))
    pairs = vectorizer.get_feature_names()
    npaDescription = dfType[['description', 'index']]
    for i, description in npaDescription.iterrows():      
        try:
            get_value(str(description.values[0]), i)
        except Exception:
            continue
    for word in stopWords:
        bagOfWords[word] = -10000000
    bagOfWords['new'] = randint(100, 2500) 
    bagOfWords['world'] = randint(100, 250000)  
    bagOfWords['never'] = randint(100, 25000)
    bagOfWords['life'] = randint(100, 25000)
    bagOfWords['look'] = randint(100, 25000)
    bagOfWords['like'] = randint(100, 25000)
    bagOfWords['time'] = randint(100, 25000)
    bagOfWords['make'] = randint(100, 25000)
    magicWords[brand_type] = deepcopy(bagOfWords)
    listOfPairs[brand_type] = pairs  



In [10]:
bestWords = {'fashion' : {}, 'beauty' : {}, 'electronics' : {}}
print(bagOfWords['nike'])
finalWords = {'fashion' : {}, 'beauty' : {}, 'electronics' : {}}
for brand_type in ['fashion', 'beauty', 'electronics']:
    for pair in listOfPairs[brand_type]:
        value = 0
        try: 
            pairs = pair.split()
            if magicWords[brand_type].get(pairs[0]) != None and magicWords[brand_type].get(pairs[1]) and magicWords[brand_type].get(pairs[2]) != None:
                value = magicWords[brand_type][pairs[0]] + magicWords[brand_type][pairs[1]] + magicWords[brand_type][pairs[2]]
            if value != 0:
                bestWords[brand_type][pair] = value
#                 print(bestWords[brand_type][pair])
        except Exception:
            if magicWords[brand_type].get(pair) != None:
                value = magicWords[brand_type][pair]
            if value != 0:
                bestWords[brand_type][pair] = value
    finalWords[brand_type] = sorted(bestWords[brand_type].items(), key=lambda x:-x[1])[:50]
    finalWords[brand_type] = [{x[0] : x[1]} for x in finalWords[brand_type]]
# print(bagOfWords['new']
# #     print(bestWords[brand_type].items()) 
# #     print(bestWords[brand_type])
finalWords

-10000000


{'fashion': [{'street sign high': 2480201.436928911},
  {'high performance available': 2343560.042872107},
  {'next level tag': 2184344.31995159},
  {'post tag friends': 2059758.6410878522},
  {'run chicago prove': 1997756.9932824862},
  {'style featuring high': 1995017.6479848279},
  {'guess tag friends': 1993082.636002635},
  {'tag three friends': 1955852.7309267353},
  {'see sport show': 1911637.9723935947},
  {'game tag friend': 1888752.279967541},
  {'sign high fives': 1801585.4039640203},
  {'top high school': 1797168.4320464465},
  {'high performance footwear': 1789759.2516198556},
  {'equitable future tag': 1760047.0672081937},
  {'next 10 going': 1738022.3178804663},
  {'presents sky high': 1714576.7602522513},
  {'high heat counts': 1698814.326883282},
  {'post tag three': 1686495.323535117},
  {'day first run': 1684814.580039428},
  {'classic tag someone': 1677126.2121259978},
  {'got louder tag': 1671972.1825477385},
  {'quick nikefuel check': 1656277.2690577707},
  {'high 

In [13]:
# finalWords
with open('topWords.json', 'w') as fp:
    json.dump(finalWords, fp)